## Validation - Analytics contents below:
- Cost estimation Queries, Analytics/Embed Timestamp Test Queries.
- Embedding audit and execution validation. References timestamp wise embedding counts, etc.
- Vector–meta parity and uniqueness, vector-length sanity and a staleness audit scan query.

### Cost tracking:
- Test 1st time 3 words - Cost: ~$0.0000005
- Previous runs: 5 × $0.0048 = ~$0.015
- Test 2 company run: $0.0288
- 2 Companies, 2019 20, Cost: $0.0475.
- API broke, gai error: Cost: 240 batches. $0.08-$0.16.
- 203k sentences, $0.7677.

```
Filters:
  CIKs: [34088, 59478, 104169, 200406, 320193, 789019, 813762, 814585, 890926, 909832, 1018724, 1045810, 1065280, 1141391, 1273813, 1276520, 1318605, 1326801, 1341439, 1403161, 1652044]
  Years: [2015, 2016, 2017, 2018, 2019, 2020]

-  Total selected: 203,084 sentences. Valid (≤1000 tokens): 203,076. 
-  Total tokens: 7,677,121. Avg tokens/sentence: 37.8. Min tokens: 1. Max tokens: 924.

```


- **COST**: Estimated cost: $0.7677, 
- Estimated batches: ~2116. Estimated time: ~4232 seconds.
- Other errors:``` - <botocore.awsrequest.AWSHTTPSConnection object at 0x0000019A01BE1F50>: Failed to resolve 'bedrock-runtime.us-east-1.amazonaws.com' ([Errno 11001] getaddrinfo failed) ```



### Quick Summary Query:

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / 'loaders'))

from ml_config_loader import MLConfig
import polars as pl

# Configure Polars display options
pl.Config.set_tbl_rows(50)              # Show up to 50 rows (default: 10)
pl.Config.set_tbl_cols(20)              # Show up to 20 columns (default: 8)
pl.Config.set_fmt_str_lengths(200)      # Show up to 200 chars per string (default: 50)
pl.Config.set_tbl_width_chars(1000)     # Total table width in chars

# For truly unlimited (use with caution on large datasets):
# pl.Config.set_tbl_rows(-1)              # Show ALL rows



config = MLConfig()

# Load meta table
cache_file = Path.cwd().parent / 'data_cache' / 'meta_embeds' / 'finrag_fact_sentences_meta_embeds.parquet'

if cache_file.exists():
    df_meta = pl.read_parquet(cache_file)
else:
    meta_uri = f"s3://{config.bucket}/{config.meta_embeds_path}"
    df_meta = pl.read_parquet(meta_uri, storage_options=config.get_storage_options())

# Company-level analytics
company_summary = df_meta.group_by(['cik_int', 'name']).agg([
    pl.col('sentenceID').n_unique().alias('distinct_sentences'),
    pl.col('report_year').n_unique().alias('distinct_years'),
    pl.col('report_year').unique().sort().cast(pl.Utf8).str.join(', ').alias('years_list')
]).sort('distinct_sentences', descending=True)

company_summary

[DEBUG] ✓ Found ModelPipeline via file path: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline
[DEBUG] ✓ AWS credentials loaded from aws_credentials.env


cik_int,name,distinct_sentences,distinct_years,years_list
i32,str,u32,u32,str
1276520,"""GENWORTH FINANCIAL INC""",49845,19,"""2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024"""
1273813,"""ASSURED GUARANTY LTD""",42732,19,"""2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024"""
813762,"""ICAHN ENTERPRISES L.P.""",36905,19,"""2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024"""
890926,"""RADIAN GROUP INC""",35757,19,"""2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024"""
814585,"""MBIA INC""",29796,19,"""2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024"""
1141391,"""Mastercard Inc""",23019,19,"""2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024"""
1341439,"""ORACLE CORP""",22724,20,"""2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025"""
1318605,"""Tesla, Inc.""",21943,15,"""2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024"""
1403161,"""VISA INC.""",21893,17,"""2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024"""


## Cost estimation Queries, Analytics/Embed Timestamp Test Queries.

In [2]:
# ============================================================================
# COST ESTIMATION: Preview embedding costs before running pipeline
# ============================================================================

import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / 'loaders'))

from ml_config_loader import MLConfig
import polars as pl

# Initialize config
config = MLConfig()

# Load meta table (use cache if available)
cache_file = Path.cwd().parent / 'data_cache' / 'meta_embeds' / 'finrag_fact_sentences_meta_embeds.parquet'

if cache_file.exists():
    df_meta = pl.read_parquet(cache_file)
else:
    meta_uri = f"s3://{config.bucket}/{config.meta_embeds_path}"
    df_meta = pl.read_parquet(meta_uri, storage_options=config.get_storage_options())

print("="*70)
print("EMBEDDING COST ESTIMATION")
print("="*70)
print(f"Mode: {config.embedding_mode}")

# Apply same filtering logic from pipeline
mode = config.embedding_mode

if mode == "full":
    df_filtered = df_meta
    print(f"\nFilter: ALL sentences")
    
elif mode == "parameterized":
    cik_filter = config.filter_cik
    year_filter = config.filter_year
    
    # Ensure lists
    cik_list = cik_filter if isinstance(cik_filter, list) else [cik_filter]
    year_list = year_filter if isinstance(year_filter, list) else [year_filter]
    
    print(f"\nFilters:")
    print(f"  CIKs: {cik_list}")
    print(f"  Years: {year_list}")
    
    df_filtered = df_meta.filter(
        pl.col('cik_int').is_in(cik_list) &
        pl.col('report_year').is_in(year_list)
    )

# Filter outliers (same as pipeline)
MAX_TOKENS_PER_SENTENCE = 1000

df_valid = df_filtered.filter(pl.col('sentence_token_count') <= MAX_TOKENS_PER_SENTENCE)
df_skipped = df_filtered.filter(pl.col('sentence_token_count') > MAX_TOKENS_PER_SENTENCE)

# Calculate costs
total_sentences = len(df_filtered)
valid_sentences = len(df_valid)
skipped_sentences = len(df_skipped)

total_tokens = df_valid['sentence_token_count'].sum()
cost_per_1k = config.get_cost_per_1k()
estimated_cost = total_tokens / 1000 * cost_per_1k

# Batch estimation
avg_tokens = df_valid['sentence_token_count'].mean()
estimated_batches = (valid_sentences + 95) // 96  # Ceiling division

print(f"\n[Selection Summary]")
print(f"  Total selected: {total_sentences:,} sentences")
print(f"  Valid (≤1000 tokens): {valid_sentences:,}")
print(f"  Skipped (>1000 tokens): {skipped_sentences:,}")

print(f"\n[Token Analysis]")
print(f"  Total tokens: {total_tokens:,}")
print(f"  Avg tokens/sentence: {avg_tokens:.1f}")
print(f"  Min tokens: {df_valid['sentence_token_count'].min()}")
print(f"  Max tokens: {df_valid['sentence_token_count'].max()}")

print(f"\n[Cost Estimate]")
print(f"  Rate: ${cost_per_1k:.5f} per 1K tokens")
print(f"  Estimated cost: ${estimated_cost:.4f}")
print(f"  Estimated batches: ~{estimated_batches}")
print(f"  Estimated time: ~{estimated_batches * 2} seconds")


[DEBUG] ✓ Found ModelPipeline via file path: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline
[DEBUG] ✓ AWS credentials loaded from aws_credentials.env
EMBEDDING COST ESTIMATION
Mode: parameterized

Filters:
  CIKs: [1276520, 1318605, 1326801, 1341439, 1403161, 1652044]
  Years: [2015, 2016, 2017, 2018, 2019, 2020]

[Selection Summary]
  Total selected: 69,512 sentences
  Valid (≤1000 tokens): 69,507
  Skipped (>1000 tokens): 5

[Token Analysis]
  Total tokens: 2,693,551
  Avg tokens/sentence: 38.8
  Min tokens: 1
  Max tokens: 924

[Cost Estimate]
  Rate: $0.00010 per 1K tokens
  Estimated cost: $0.2694
  Estimated batches: ~725
  Estimated time: ~1450 seconds


In [3]:
print(f"\n[Breakdown by Company]")
company_stats = df_valid.group_by(['cik_int', 'name']).agg([
    pl.len().alias('sentences'),
    pl.col('sentence_token_count').sum().alias('tokens')
]).with_columns([
    (pl.col('tokens') / 1000 * cost_per_1k).alias('cost')
]).sort('sentences', descending=True)

company_stats



[Breakdown by Company]


cik_int,name,sentences,tokens,cost
i32,str,u32,i64,f64
1276520,"""GENWORTH FINANCIAL INC""",20442,810332,0.0810332
1318605,"""Tesla, Inc.""",11794,476593,0.0476593
1652044,"""Alphabet Inc.""",10275,329555,0.032956
1403161,"""VISA INC.""",9994,346360,0.034636
1341439,"""ORACLE CORP""",9297,413701,0.04137
1326801,"""Meta Platforms, Inc.""",5787,243356,0.0243356
1326801,"""Facebook Inc""",1918,73654,0.0073654


## EMBEDDING AUDIT - Execution History


In [4]:
import polars as pl
from pathlib import Path

# Load both tables
cache_dir = Path.cwd().parent / 'data_cache' / 'meta_embeds'
df_meta = pl.read_parquet(cache_dir / 'finrag_fact_sentences_meta_embeds.parquet')

# Load the lean vector table (has sentenceID + embedding_id + embedding)
embedding_dir = Path.cwd().parent / 'data_cache' / 'embeddings' / 'cohere_1024d'
df_vectors = pl.read_parquet(embedding_dir / 'finrag_embeddings_cohere_1024d.parquet')

print(f"Meta table: {df_meta.shape}")
print(f"Vector table: {df_vectors.shape}")
print(f"\nVector table columns: {df_vectors.columns}")

# Join meta with vectors (LEFT JOIN to keep all meta rows)
df_joined = df_meta.join(
    df_vectors.select(['sentenceID', 'embedding_id', 'embedding']),
    on=['sentenceID', 'embedding_id'],
    how='left'
)

print(f"Joined table: {df_joined.shape}")

# Create embedded flag
df_with_flag = df_joined.with_columns([
    pl.when(pl.col('embedding_id').is_not_null())
      .then(pl.lit('Yes'))
      .otherwise(pl.lit('No'))
      .alias('embedded')
])

# Audit summary with embedding dimension checks
audit_summary = df_with_flag.group_by(['embedded', 'embedding_id', 'embedding_date']).agg([
    pl.len().alias('sentences'),
    pl.col('cik_int').n_unique().alias('distinct_companies'),
    pl.col('report_year').n_unique().alias('distinct_years'),
    pl.col('name').unique().sort().str.join(', ').alias('companies'),
    pl.col('report_year').unique().sort().cast(pl.Utf8).str.join(', ').alias('years'),
    
    # Embedding dimension checks (only for embedded rows)
    pl.col('embedding').list.len().n_unique().alias('distinct_dims'),
    pl.col('embedding').list.len().min().alias('min_dim'),
    pl.col('embedding').list.len().max().alias('max_dim'),
]).sort('embedding_date', descending=True, nulls_last=True)

print("="*70)
print("EMBEDDING AUDIT - Execution History")
print("="*70)

print(audit_summary)

: 

In [3]:

import polars as pl

# Configure Polars display options
pl.Config.set_tbl_rows(50)              # Show up to 50 rows (default: 10)
pl.Config.set_tbl_cols(20)              # Show up to 20 columns (default: 8)
pl.Config.set_fmt_str_lengths(200)      # Show up to 200 chars per string (default: 50)
pl.Config.set_tbl_width_chars(1000)     # Total table width in chars


# Overall stats
total_embedded = df_with_flag.filter(pl.col('embedded') == 'Yes').shape[0]
total_not_embedded = df_with_flag.filter(pl.col('embedded') == 'No').shape[0]

print(f"\n[Overall Statistics]")
print(f"  ✓ Embedded: {total_embedded:,} sentences")
print(f"  ○ Not embedded: {total_not_embedded:,} sentences")
print(f"  Coverage: {total_embedded/(total_embedded+total_not_embedded)*100:.2f}%")


### distinct years per company with embeddings
company_years = df_with_flag.filter(pl.col('embedded') == 'Yes').group_by(['cik_int', 'name']).agg([
    pl.col('report_year').unique().sort().cast(pl.Utf8).str.join(', ').alias('embedded_years'),
    pl.col('report_year').n_unique().alias('num_embedded_years')
]).sort('num_embedded_years', descending=True)  

company_years


[Overall Statistics]
  ✓ Embedded: 203,076 sentences
  ○ Not embedded: 266,176 sentences
  Coverage: 43.28%


cik_int,name,embedded_years,num_embedded_years
i32,str,str,u32
1018724,"""AMAZON COM INC""","""2015, 2016, 2017, 2018, 2019, 2020""",6
34088,"""EXXON MOBIL CORP""","""2015, 2016, 2017, 2018, 2019, 2020""",6
1318605,"""Tesla, Inc.""","""2015, 2016, 2017, 2018, 2019, 2020""",6
1045810,"""NVIDIA CORP""","""2015, 2016, 2017, 2018, 2019, 2020""",6
1141391,"""Mastercard Inc""","""2015, 2016, 2017, 2018, 2019, 2020""",6
890926,"""RADIAN GROUP INC""","""2015, 2016, 2017, 2018, 2019, 2020""",6
1276520,"""GENWORTH FINANCIAL INC""","""2015, 2016, 2017, 2018, 2019, 2020""",6
104169,"""Walmart Inc.""","""2015, 2016, 2017, 2018, 2019, 2020""",6
1652044,"""Alphabet Inc.""","""2015, 2016, 2017, 2018, 2019, 2020""",6


In [ ]:
"""

T1: 
embedded	embedding_id	embedding_date	sentences	distinct_companies	distinct_years	companies	years
str	str	datetime[μs]	u32	u32	u32	str	str
"Yes"	"bedrock_cohere_v4_1024d_20251108_1337"	2025-11-08 13:37:18.036329	7764	2	2	"MICROSOFT CORP, Tesla, Inc."	"2016, 2017"
"Yes"	"bedrock_cohere_v4_1024d_20251107_1206"	2025-11-07 12:06:53.647536	1202	1	1	"Apple Inc."	"2016"


T2:
embedded	embedding_id	embedding_date	sentences	distinct_companies	distinct_years	companies	years
str	str	datetime[μs]	u32	u32	u32	str	str
"Yes"	"bedrock_cohere_v4_1024d_20251108_1350"	2025-11-08 13:50:57.176125	1202	1	1	"Apple Inc."	"2016"
"Yes"	"bedrock_cohere_v4_1024d_20251108_1337"	2025-11-08 13:37:18.036329	7764	2	2	"MICROSOFT CORP, Tesla, Inc."	"2016, 2017"


T3:
embedded	embedding_id	embedding_date	sentences	distinct_companies	distinct_years	companies	years
str	str	datetime[μs]	u32	u32	u32	str	str
"Yes"	"bedrock_cohere_v4_1024d_20251109_0457"	2025-11-09 04:57:48.169899	13152	2	2	"EXXON MOBIL CORP, GENWORTH FINANCIAL INC"	"2019, 2020"
"Yes"	"bedrock_cohere_v4_1024d_20251108_1501"	2025-11-08 15:01:53.445619	1202	1	1	"Apple Inc."	"2016"
"Yes"	"bedrock_cohere_v4_1024d_20251108_1337"	2025-11-08 13:37:18.036329	7764	2	2	"MICROSOFT CORP, Tesla, Inc."	"2016, 2017"


"""

## Vector–meta parity and uniqueness, vector-length sanity and a staleness scan query.

- Integrity + Staleness Audit 
- Reads cached Stage-2 meta
- Auto-discovers cached embeddings parquet(s) under data_cache/embeddings/*/
- Checks: vector–meta parity, uniqueness, vector length vs dims, and staleness per (CIK, year)
- Every sentence that has an embedding vector stored in the vectors table should have a matching, correctly populated metadata record in the meta table — and vice-versa.
  

1. Missing in meta – df_vectors.join(df_meta, how="anti") finds any vector rows whose sentenceID isn’t in the meta table.
2. Duplicate vectors – group_by(sentenceID).len()>1. Ensures exactly one vector per sentence.
3. Null meta fields – For any sentence that has a vector, make sure all its meta fields (embedding_id, model, dims, date) are filled.
4. Mismatched vector length – Compares len(embedding) with embedding_dims. → Catches broken writes or dimension drift.


In [14]:

import polars as pl
from pathlib import Path

ROOT = Path.cwd().parent
META_PATH = ROOT / "data_cache" / "meta_embeds" / "finrag_fact_sentences_meta_embeds.parquet"
EMB_ROOT = ROOT / "data_cache" / "embeddings"

print("="*70)
print("INTEGRITY + STALENESS AUDIT")
print("="*70)

# ----- Load meta -----
df_meta = pl.read_parquet(META_PATH)


COL_SENT_ID   = "sentenceID"
COL_CIK       = "cik_int"        
COL_YEAR      = "report_year"     
COL_COMPANY   = "name"           
COL_EID       = "embedding_id"
COL_EMODEL    = "embedding_model"
COL_EDIMS     = "embedding_dims"
COL_EDATE     = "embedding_date"

# ----- Load vectors (auto-discover all providers under embeddings/*/*.parquet) -----
vec_files = sorted(EMB_ROOT.glob("*/*.parquet"))
if not vec_files:
    raise FileNotFoundError(f"No local vector parquet files found under {EMB_ROOT}")

dfs = []
for p in vec_files:
    provider = p.parent.name  # folder name (e.g., 'cohere_1024d')
    dfp = pl.read_parquet(p).with_columns(pl.lit(provider).alias("__provider"))
    dfs.append(dfp)
df_vectors = pl.concat(dfs, how="vertical_relaxed")

print(f"[LOAD] meta rows = {df_meta.height:,} | vectors rows = {df_vectors.height:,} | files = {len(vec_files)}")
print("       providers:", ", ".join(sorted(set(df_vectors['__provider'].to_list()))))

# =========================
# A) PARITY / UNIQUENESS
# =========================

# 1) Every vector sentenceID must appear in meta
missing_in_meta = (
    df_vectors.join(df_meta.select([COL_SENT_ID]), on=COL_SENT_ID, how="anti").height
)

# 2) No duplicate vectors per sentenceID
dupe_vectors = (
    df_vectors.group_by(COL_SENT_ID).len().filter(pl.col("len") > 1).height
)

# 3) Meta fields must be non-null for sentences that have vectors
null_meta_fields = df_meta.filter(
    pl.col(COL_SENT_ID).is_in(df_vectors[COL_SENT_ID])
    & (
        pl.col(COL_EID).is_null()
        | pl.col(COL_EMODEL).is_null()
        | pl.col(COL_EDIMS).is_null()
        | pl.col(COL_EDATE).is_null()
    )
).height

# 4) Vector list length must equal meta embedding_dims
vec_lens = df_vectors.select([
    pl.col(COL_SENT_ID),
    pl.col("embedding").list.len().alias("__len")
])
lens_chk = vec_lens.join(
    df_meta.select([COL_SENT_ID, COL_EDIMS, COL_EMODEL]),
    on=COL_SENT_ID, how="left"
)
mismatch_len = lens_chk.filter(pl.col("__len") != pl.col(COL_EDIMS)).height

print("\n[PARITY]")
print(f"  vectors: {df_vectors.height:,}")
print(f"  missing_in_meta: {missing_in_meta}")
print(f"  duplicate_vectors_per_sentenceID: {dupe_vectors}")
print(f"  null_meta_fields_for_vector_rows: {null_meta_fields}")
print(f"  mismatched_vector_length_vs_dims: {mismatch_len}")

# Optional: show per-model vector shape stats
by_model = lens_chk.group_by(COL_EMODEL).agg([
    pl.len().alias("rows"),
    pl.col("__len").min().alias("min_len"),
    pl.col("__len").max().alias("max_len")
]).sort("rows", descending=True)
print("\n[SHAPE by model]")
print(by_model)

# =========================
# B) LATEST-WINS STALENESS
# =========================
# For each (CIK, year), all embedded rows should carry the max embedding_date for that key.

embedded = df_meta.filter(pl.col(COL_EID).is_not_null())
max_dates = embedded.group_by([COL_CIK, COL_YEAR]).agg(
    pl.max(COL_EDATE).alias("__max_date")
)

stale = embedded.join(max_dates, on=[COL_CIK, COL_YEAR], how="inner").filter(
    pl.col(COL_EDATE) < pl.col("__max_date")
)

print("\n[LATEST-WINS]")
print(f"  stale_rows: {stale.height}")
if stale.height:
    print("  sample (up to 10):")
    print(
        stale.select([COL_SENT_ID, COL_COMPANY, COL_CIK, COL_YEAR, COL_EDATE, "__max_date"])
             .sort([COL_COMPANY, COL_YEAR, COL_EDATE])
             .head(10)
    )

# High-level summary by (company, year): how many rows are at latest timestamp vs stale
if embedded.height:
    latest_flag = embedded.join(max_dates, on=[COL_CIK, COL_YEAR], how="left") \
        .with_columns((pl.col(COL_EDATE) == pl.col("__max_date")).alias("__is_latest"))

    # Sum the expression directly; don't wrap an Expr inside pl.sum(...)
    rollup = latest_flag.group_by([COL_COMPANY, COL_YEAR]).agg([
        pl.col("__is_latest").cast(pl.Int32).sum().alias("rows_latest"),
        (pl.len() - pl.col("__is_latest").cast(pl.Int32).sum()).alias("rows_stale"),
    ]).sort([COL_COMPANY, COL_YEAR])

    print("\n[LATEST-WINS SUMMARY per company/year]")
    print(rollup)


print("\n✓ Audit complete.")


INTEGRITY + STALENESS AUDIT
[LOAD] meta rows = 469,252 | vectors rows = 203,076 | files = 1
       providers: cohere_1024d


C:\Users\joems\AppData\Local\Temp\ipykernel_23764\3361716688.py:55: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  null_meta_fields = df_meta.filter(



[PARITY]
  vectors: 203,076
  missing_in_meta: 0
  duplicate_vectors_per_sentenceID: 0
  null_meta_fields_for_vector_rows: 0
  mismatched_vector_length_vs_dims: 0

[SHAPE by model]
shape: (1, 4)
┌───────────────────┬────────┬─────────┬─────────┐
│ embedding_model   ┆ rows   ┆ min_len ┆ max_len │
│ ---               ┆ ---    ┆ ---     ┆ ---     │
│ str               ┆ u32    ┆ u32     ┆ u32     │
╞═══════════════════╪════════╪═════════╪═════════╡
│ cohere.embed-v4:0 ┆ 203076 ┆ 1024    ┆ 1024    │
└───────────────────┴────────┴─────────┴─────────┘

[LATEST-WINS]
  stale_rows: 0

[LATEST-WINS SUMMARY per company/year]
shape: (124, 4)
┌────────────────┬─────────────┬─────────────┬────────────┐
│ name           ┆ report_year ┆ rows_latest ┆ rows_stale │
│ ---            ┆ ---         ┆ ---         ┆ ---        │
│ str            ┆ i64         ┆ i32         ┆ i64        │
╞════════════════╪═════════════╪═════════════╪════════════╡
│ AMAZON COM INC ┆ 2015        ┆ 235         ┆ 0          │


## S3 vect cost analytics

In [5]:
# S3 Vectors cost model — 

# === STUDENT PROJECT ASSUMPTIONS ===
N_total_vectors = 1_000_000        # Full dataset
dims = 1024                        # Cohere v4 embeddings
avg_key_bytes = 12                 # Short int key (not string)
filter_int_fields = 3              # cik_int (4B), year (2B), section (2B) = 8B
filter_other_bytes = 1016          # Pad to 1KB total filterable
nonfilter_bytes = 512              # embedding_id, minimal snippet

# Sharding strategy
N_per_index = 100_000              # Typical shard size (CIK × Year)
monthly_queries = 1_000            # Student demo usage
monthly_refresh_rate = 1/6         # One full refresh every 6 months

# === AWS PRICING (us-east-1) ===
storage_per_GB_month = 0.06
put_per_GB = 0.20
api_per_million_queries = 2.50
tier1_per_TB = 0.004               # First 100K vectors
tier2_per_TB = 0.002               # Above 100K vectors

# === CALCULATIONS ===
KB, MB, GB, TB = 1024, 1024**2, 1024**3, 1024**4

# Per-vector sizes
vec_bytes = 4 * dims
key_bytes = avg_key_bytes
filter_bytes = filter_int_fields * 1 + filter_other_bytes
storage_bytes = vec_bytes + key_bytes + filter_bytes + nonfilter_bytes
query_bytes = vec_bytes + key_bytes + filter_bytes  # No non-filterable

# Storage cost
storage_GB = (N_total_vectors * storage_bytes) / GB
storage_cost = storage_GB * storage_per_GB_month

# PUT cost (amortized monthly)
put_cost_monthly = (storage_GB * put_per_GB) * monthly_refresh_rate

# Query cost
tier1_vecs = min(N_per_index, 100_000)
tier2_vecs = max(N_per_index - 100_000, 0)

tier1_TB = (tier1_vecs * query_bytes) / TB
tier2_TB = (tier2_vecs * query_bytes) / TB

data_cost_per_query = (tier1_TB * tier1_per_TB) + (tier2_TB * tier2_per_TB)
api_cost_per_query = api_per_million_queries / 1_000_000

query_cost_monthly = monthly_queries * (data_cost_per_query + api_cost_per_query)

total_monthly = storage_cost + put_cost_monthly + query_cost_monthly

# === REPORT ===
print("=" * 70)
print("S3 VECTORS COST MODEL")
print("=" * 70)
print(f"Vectors: {N_total_vectors:,} | Dims: {dims} | Shard size: {N_per_index:,}")
print(f"Storage per vector: {storage_bytes/KB:.2f} KB | Query per vector: {query_bytes/KB:.2f} KB")
print(f"\nCOSTS:")
print(f"  Storage:  {storage_GB:.2f} GB @ ${storage_per_GB_month}/GB → ${storage_cost:.2f}/mo")
print(f"  PUT:      {storage_GB:.2f} GB × 1/6 refresh @ ${put_per_GB}/GB → ${put_cost_monthly:.2f}/mo")
print(f"  Queries:  {monthly_queries:,} queries/mo")
print(f"    - Data/query: Tier1={tier1_TB:.6f}TB @ ${tier1_per_TB}/TB, Tier2={tier2_TB:.6f}TB @ ${tier2_per_TB}/TB")
print(f"    - API/query: ${api_cost_per_query:.8f}")
print(f"    - Total query cost: ${query_cost_monthly:.4f}/mo")
print(f"\nTOTAL MONTHLY: ${total_monthly:.2f}")
print("=" * 70)

# Comparison
print(f"\nCOMPARISON:")
print(f"  S3 Vectors (sharded):  ${total_monthly:.2f}/mo")
print(f"  Current Parquet+NumPy: $0.14/mo")
print(f"  Difference: +${total_monthly - 0.14:.2f}/mo")
print(f"  Pinecone/Qdrant:       $70-300/mo")
print("=" * 70)

S3 VECTORS COST MODEL
Vectors: 1,000,000 | Dims: 1024 | Shard size: 100,000
Storage per vector: 5.51 KB | Query per vector: 5.01 KB

COSTS:
  Storage:  5.25 GB @ $0.06/GB → $0.32/mo
  PUT:      5.25 GB × 1/6 refresh @ $0.2/GB → $0.18/mo
  Queries:  1,000 queries/mo
    - Data/query: Tier1=0.000466TB @ $0.004/TB, Tier2=0.000000TB @ $0.002/TB
    - API/query: $0.00000250
    - Total query cost: $0.0044/mo

TOTAL MONTHLY: $0.49

COMPARISON:
  S3 Vectors (sharded):  $0.49/mo
  Current Parquet+NumPy: $0.14/mo
  Difference: +$0.35/mo
  Pinecone/Qdrant:       $70-300/mo


In [7]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / 'loaders'))

from ml_config_loader import MLConfig
import polars as pl

# Configure Polars display options
pl.Config.set_tbl_rows(50)              # Show up to 50 rows (default: 10)
pl.Config.set_tbl_cols(20)              # Show up to 20 columns (default: 8)
pl.Config.set_fmt_str_lengths(200)      # Show up to 200 chars per string (default: 50)
pl.Config.set_tbl_width_chars(1000)     # Total table width in chars

# For truly unlimited (use with caution on large datasets):
# pl.Config.set_tbl_rows(-1)              


config = MLConfig()

# Load meta table
cache_file = Path.cwd().parent / 'data_cache' / 'stage1_facts' / 'finrag_fact_sentences.parquet'

if cache_file.exists():
    df_meta = pl.read_parquet(cache_file)
else:
    meta_uri = f"s3://{config.bucket}/{config.meta_embeds_path}"
    df_meta = pl.read_parquet(meta_uri, storage_options=config.get_storage_options())

# Company-level analytics
company_summary = df_meta.group_by(['cik_int', 'name', 'load_method']).agg([
    pl.col('sentenceID').n_unique().alias('distinct_sentences'),
    pl.col('report_year').n_unique().alias('distinct_years'),
    pl.col('temporal_bin').n_unique().alias('distinct_temporal_bins'),
    # pl.col('load_method').
    pl.col('report_year').unique().sort().cast(pl.Utf8).str.join(', ').alias('years_list')
]).sort('cik_int', 'name', 'load_method', 'distinct_sentences', descending=True)

company_summary

[DEBUG] ✓ AWS credentials loaded from aws_credentials.env


cik_int,name,load_method,distinct_sentences,distinct_years,distinct_temporal_bins,years_list
i32,str,str,u32,u32,u32,str
1652044,"""Alphabet Inc.""","""incremental_inject""",8510,5,2,"""2015, 2016, 2017, 2018, 2019"""
1652044,"""Alphabet Inc.""","""extract_and_convert""",8767,5,2,"""2020, 2021, 2022, 2023, 2024"""
1403161,"""VISA INC.""","""stratified_sampling""",14261,13,3,"""2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020"""
1403161,"""VISA INC.""","""extract_and_convert""",7632,4,1,"""2021, 2022, 2023, 2024"""
1341439,"""ORACLE CORP""","""stratified_sampling""",13964,15,3,"""2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020"""
1341439,"""ORACLE CORP""","""extract_and_convert""",8760,5,1,"""2021, 2022, 2023, 2024, 2025"""
1326801,"""Meta Platforms, Inc.""","""stratified_sampling""",6900,9,2,"""2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020"""
1326801,"""Meta Platforms, Inc.""","""extract_and_convert""",8446,4,1,"""2021, 2022, 2023, 2024"""
1326801,"""Facebook Inc""","""extract_and_convert""",1918,1,1,"""2020"""


In [14]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / 'loaders'))

from ml_config_loader import MLConfig
import polars as pl



config = MLConfig()

# Load meta table
cache_file = Path.cwd().parent / 'data_cache' / 'stage1_facts' / 'finrag_fact_sentences.parquet'

if cache_file.exists():
    df_meta = pl.read_parquet(cache_file)
else:
    meta_uri = f"s3://{config.bucket}/{config.meta_embeds_path}"
    df_meta = pl.read_parquet(meta_uri, storage_options=config.get_storage_options())


distinct_cik = df_meta.select(pl.col('cik_int').unique().sort()).to_series()

dist_cik2 = df_meta.select(pl.col('cik_int').unique().sort().cast(pl.UInt64).str.join(', '))

print(dist_cik2)

[DEBUG] ✓ AWS credentials loaded from aws_credentials.env
shape: (1, 1)
┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ cik_int                                                                                                                                                                         │
│ ---                                                                                                                                                                             │
│ str                                                                                                                                                                             │
╞═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ 34088, 59478, 104169, 2004

In [ ]:
## Checking section name.


import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / 'loaders'))

from ml_config_loader import MLConfig
import polars as pl

# Configure Polars display options
pl.Config.set_tbl_rows(50)              # Show up to 50 rows (default: 10)
pl.Config.set_tbl_cols(20)              # Show up to 20 columns (default: 8)
pl.Config.set_fmt_str_lengths(200)      # Show up to 200 chars per string (default: 50)
pl.Config.set_tbl_width_chars(1000)     # Total table width in chars

# For truly unlimited (use with caution on large datasets):
# pl.Config.set_tbl_rows(-1)              


config = MLConfig()

# Load meta table
cache_file = Path.cwd().parent / 'data_cache' / 'stage1_facts' / 'finrag_fact_sentences.parquet'

if cache_file.exists():
    df_meta = pl.read_parquet(cache_file)
else:
    meta_uri = f"s3://{config.bucket}/{config.meta_embeds_path}"
    df_meta = pl.read_parquet(meta_uri, storage_options=config.get_storage_options())

company_summary = df_meta.group_by(['cik_int', 'name', 'section_name']).agg([
    pl.col('sentenceID').n_unique().alias('distinct_sentences'),
]).sort('cik_int', 'name', 'section_name', 'distinct_sentences', descending=True)

company_summary

[DEBUG] ✓ AWS credentials loaded from aws_credentials.env


cik_int,name,section_name,distinct_sentences
i32,str,str,u32
1652044,"""Alphabet Inc.""","""ITEM_9B""",61
1652044,"""Alphabet Inc.""","""ITEM_9A""",141
1652044,"""Alphabet Inc.""","""ITEM_9""",20
1652044,"""Alphabet Inc.""","""ITEM_8""",6644
1652044,"""Alphabet Inc.""","""ITEM_7A""",494
1652044,"""Alphabet Inc.""","""ITEM_7""",3473
1652044,"""Alphabet Inc.""","""ITEM_6""",36
1652044,"""Alphabet Inc.""","""ITEM_5""",313
1652044,"""Alphabet Inc.""","""ITEM_4""",27
